In [13]:
from bs4 import BeautifulSoup
import os
import json
import utils
import pandas as pd

In [14]:
if not os.path.exists(os.path.join(".", "news_data.csv")):
    news_data = pd.DataFrame(data = {"link": []})
    news_data.to_csv(os.path.join(".", "news_data.csv"))
else:
    news_data = pd.read_csv(os.path.join(".", "news_data.csv"), index_col=0)

In [15]:
news_data.head()
news_data.shape

(9130, 1)

In [25]:
list_webs = json.load(open(os.path.join(".", "list_webs.json"), encoding='UTF-8'))

In [26]:
for web_obj in list_webs:
    if not web_obj.get("status", True):
        if web_obj.get("type_crawl", -1) == 1:
            page = web_obj.get("info").get("start_from_page", 1) - 1
            while True:
                page += 1
                list_link = []
                if page == -1:
                    link = web_obj.get("path")
                else:
                    link = web_obj.get("path") + web_obj.get("info").get("prefix") + str(page) + web_obj.get("info").get("postfix")
                # print(link)
                list_link = utils.crawl_list_news(link = link, 
                                                  tag = web_obj.get("info").get("tag", None), 
                                                  attr = web_obj.get("info").get("attr", None),
                                                  domain = web_obj.get("info").get("result_domain", ""))
                if len(list_link) == 0:
                    break
                else:
                    # print(list_link)
                    # break
                    news_data = pd.concat([news_data, pd.DataFrame({"link": list_link})], ignore_index=True, axis=0)
                    news_data.drop_duplicates(ignore_index=True)
                    news_data.to_csv(os.path.join(".", "news_data.csv"))
                if page == -1 or page == web_obj.get("info").get("stop_at_page", -2):
                    break
            # break
            web_obj["status"] = True
            with open(os.path.join(".", "list_webs.json"), "w") as f:
                f.write(json.dumps(list_webs, indent = 4))
            break